In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from load_mocks import ensemble_mock
from minijpas_LF_and_puricomp import nb_or_3fm_cont

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()

In [ ]:
qso_name = 'QSO_double_train_minijpas_DR16_D_0'
sf_name = 'LAE_12.5deg_z2-4.25_train_minijpas_0'
gal_name = 'GAL_LC_minijpas_0'

pm_flx, pm_err, zspec, EW_lya, L_lya, is_qso, is_sf, is_gal, _, _, _ =\
    ensemble_mock(qso_name, gal_name, sf_name)

mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

N_sources = pm_flx.shape[1]
N_sources

In [ ]:
g = flux_to_mag(pm_flx[-3], w_central[-3])
r = flux_to_mag(pm_flx[-2], w_central[-2])
i = flux_to_mag(pm_flx[-1], w_central[-1])

fig, ax = plt.subplots(figsize=(7, 6))

color = g - i
lae_mask = (L_lya > 42) & (zspec > 2) & (zspec < 3.5)
bins = np.linspace(-5, 5, 50)
ax.hist(color[is_qso & lae_mask], label='QSO', bins=bins, histtype='step', density=True)
ax.hist(color[is_sf & lae_mask], label='SF', bins=bins, histtype='step', density=True)
ax.hist(color[is_gal], label='Galaxy', bins=bins, histtype='step', density=True)

ax.legend()

plt.show()

In [ ]:
nb_lya = NB_z(zspec)

In [ ]:
def cont_arr(pm_flx, pm_err, nb_lya):
    cont_flx_lya_nb, cont_err_lya_nb, cont_flx_oth_nb, cont_err_oth_nb =\
        nb_or_3fm_cont(pm_flx, pm_err, 'nb')
    cont_flx_lya_3f, cont_err_lya_3f, cont_flx_oth_3f, cont_err_oth_3f =\
        nb_or_3fm_cont(pm_flx, pm_err, '3fm')

    cont_lya_nb = cont_flx_lya_nb[nb_lya, np.arange(N_sources)]
    cont_lya_3f = cont_flx_lya_3f[nb_lya, np.arange(N_sources)]

    return cont_lya_nb, cont_lya_3f

cont_lya_nb, cont_lya_3f = cont_arr(pm_flx, pm_err, nb_lya)

In [ ]:
def cont_in(L_lya, EW0, z_lya):
    dL = cosmo.luminosity_distance(z_lya).to(u.cm).value
    return 10**L_lya / (4*np.pi * dL**2 * EW0 * (1 + z_lya))

In [ ]:
cont_lya_in = cont_in(L_lya, EW_lya, zspec)

In [ ]:
mask = (cont_lya_in > 1e-18) & np.isfinite(cont_lya_in) & np.isfinite(cont_lya_3f)
print(np.nanmedian(np.abs(cont_lya_in[mask] - cont_lya_3f[mask])))
print(np.nanmedian(np.abs(cont_lya_in[mask] - cont_lya_nb[mask])))

fig, ax = plt.subplots(figsize=(7, 6))

ax.scatter(cont_lya_in, cont_lya_nb, c='k', s=1)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-19, 1e-15)
ax.set_xlim(1e-19, 1e-15)
ax.set_xlabel(r'Real $f_\mathrm{cont}$', fontsize=15)
ax.set_ylabel(r'Estimated $f_\mathrm{cont}$', fontsize=15)

plt.show()

fig, ax = plt.subplots(figsize=(7, 6))

ax.scatter(cont_lya_in, cont_lya_3f, c='k', s=1)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-19, 1e-15)
ax.set_xlim(1e-19, 1e-15)
ax.set_xlabel(r'Real $f_\mathrm{cont}$', fontsize=15)
ax.set_ylabel(r'Estimated $f_\mathrm{cont}$', fontsize=15)

plt.show()

fig, ax = plt.subplots(figsize=(7, 6))

ax.scatter(cont_lya_nb, cont_lya_3f, c='k', s=1)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-19, 1e-15)
ax.set_xlim(1e-19, 1e-15)
ax.set_xlabel(r'Real $f_\mathrm{cont}$', fontsize=15)
ax.set_ylabel(r'Estimated $f_\mathrm{cont}$', fontsize=15)

plt.show()